In [1]:
from couplets_utils import *
from keras.layers import *
from keras.models import Model
from keras import backend as K
import numpy as np
import tensorflow as tf

%load_ext autoreload
%autoreload 2

/Users/zytec/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
N_A = 128

In [3]:
sample = load_sample_datasets(vocabs_size=2000, max_len=30, batch_size=16, sample_size=1000, n_a=N_A)
sample_gen = sample['sample_gen']
index2word = sample['index2word']
word2index = sample['word2index']
vocabs_size = sample['vocabs_size']
max_len = sample['max_len']

In [4]:
print('sample_gen length :', len(sample_gen))

sample_gen length : 63


In [5]:
def create_train_model(n_x, n_a, Tx, keep_prob=0.5):    
    input = Input(shape=(Tx, n_x), name='x0')
    a_0_0 = Input(shape=(n_a,), name='a_0_0')
    c_0_0 = Input(shape=(n_a,), name='c_0_0')
    a_1_0 = Input(shape=(n_a,), name='a_1_0')
    c_1_0 = Input(shape=(n_a,), name='c_1_0')
        
    a_0 = a_0_0
    c_0 = c_0_0
    a_1 = a_1_0
    c_1 = c_1_0
    
    lstm_cell_0 = LSTM(units=n_a, return_state=True, name='lstm_0')
    lstm_cell_1 = LSTM(units=n_a, return_state=True, name='lstm_1')
    dense_layer_2 = Dense(units=n_x, activation='softmax', name='softmax_2')
        
    outputs = []
        
    for i in range(Tx):
        x = Lambda(lambda j: j[:, i, :])(input)
        x = Reshape(target_shape=(1, -1))(x)
        a_0, x, c_0 = lstm_cell_0(x, initial_state=[a_0, c_0])
        x = Dropout(rate=1-keep_prob)(x)
        x = Reshape(target_shape=(1, -1))(x)
        a_1, x, c_1 = lstm_cell_1(x, initial_state=[a_1, c_1])
        x = Dropout(rate=1-keep_prob)(x)
        x = dense_layer_2(x)
        outputs.append(x)
                            
    model = Model(inputs=[input, a_0_0, c_0_0, a_1_0, c_1_0], outputs=outputs)
        
    return model

In [6]:
train_model = create_train_model(vocabs_size, N_A, max_len)

In [7]:
train_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
result = train_model.fit_generator(sample_gen, epochs=1)

Epoch 1/1
63/63 [==============================] - 65s 1s/step - loss: 157.4750 - softmax_2_loss: 2.2523 - softmax_2_acc: 0.0000e+00 - softmax_2_acc_1: 0.0000e+00 - softmax_2_acc_2: 0.0000e+00 - softmax_2_acc_3: 0.0000e+00 - softmax_2_acc_4: 9.9219e-04 - softmax_2_acc_5: 9.9219e-04 - softmax_2_acc_6: 0.0238 - softmax_2_acc_7: 0.0228 - softmax_2_acc_8: 0.0327 - softmax_2_acc_9: 0.0327 - softmax_2_acc_10: 0.0516 - softmax_2_acc_11: 0.0506 - softmax_2_acc_12: 0.1667 - softmax_2_acc_13: 0.1667 - softmax_2_acc_14: 0.1796 - softmax_2_acc_15: 0.1796 - softmax_2_acc_16: 0.6816 - softmax_2_acc_17: 0.6796 - softmax_2_acc_18: 0.6865 - softmax_2_acc_19: 0.6865 - softmax_2_acc_20: 0.7292 - softmax_2_acc_21: 0.7262 - softmax_2_acc_22: 0.7520 - softmax_2_acc_23: 0.7500 - softmax_2_acc_24: 0.7847 - softmax_2_acc_25: 0.7837 - softmax_2_acc_26: 0.9256 - softmax_2_acc_27: 0.9276 - softmax_2_acc_28: 0.9643 - softmax_2_acc_29: 0.9683 


In [9]:
train_model.save_weights('./weights.h5')

In [5]:
def create_random_infer_model(n_x, n_a, Tx):
    input = Input(shape=(n_x,), name='x_0')
    a_0_0 = Input(shape=(n_a,), name='a_0_0')
    c_0_0 = Input(shape=(n_a,), name='c_0_0')
    a_1_0 = Input(shape=(n_a,), name='a_1_0')
    c_1_0 = Input(shape=(n_a,), name='c_1_0')
    
    x = input
    a_0 = a_0_0
    c_0 = c_0_0
    a_1 = a_1_0
    c_1 = c_1_0

    lstm_cell_0 = LSTM(units=n_a, return_state=True, name='lstm_0')
    lstm_cell_1 = LSTM(units=n_a, return_state=True, name='lstm_1')
    dense_layer_2 = Dense(units=n_x, activation='softmax', name='softmax_2')
    
    def sample(x):
        return  tf.one_hot(tf.multinomial(x, 1), n_x, axis=-1)
    
    outputs = []
    
    for i in range(Tx):              
        x = Reshape(target_shape=(1, -1))(x)
        a_0, x, c_0 = lstm_cell_0(x, initial_state=[a_0, c_0])
        x = Reshape(target_shape=(1, -1))(x)
        a_1, x, c_1 = lstm_cell_1(x, initial_state=[a_1, c_1])
        x = dense_layer_2(x)
        x = Lambda(sample)(x)
        x = Reshape((-1,))(x)
        outputs.append(x)
        
    model = Model(inputs=[input, a_0_0, c_0_0, a_1_0, c_1_0], outputs=outputs)
    
    return model

In [6]:
infer_model = create_random_infer_model(vocabs_size, N_A, max_len)
infer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)
(?, 2000)
(?, ?)


In [7]:
infer_model.load_weights('./weights.h5')

In [20]:
def write_couplets(num, infer_model):
    x = np.zeros((num, vocabs_size))
    a_0 = np.zeros((num, N_A))
    c_0 = np.zeros((num, N_A))
    a_1 = np.zeros((num, N_A))
    c_1 = np.zeros((num, N_A))
    
    result = infer_model.predict([x, a_0, c_0, a_1, c_1])
    result_text = convert_onehots_to_text(np.array(result), index2word)
    
    return result_text

In [21]:
text = '天天向上'
result_text = write_couplets(3, infer_model)
print(result_text)

['章谓鲜溢敬肠板语酸暖白翠人锁那碑剪傲勿泰庆栏沁勾吃轻拥昼亭阁', '选材凭愚标材慨嫦欢服柏哲拜洪聊知愁复垒面醒圆瓯备赋新泻总贪妨', '狼立软温帅扬比并庙暗补助培效赏生认庶宛屋酿设侠蒙奔辱互凭龟航']
